In [1]:
from lxml import html
import requests
import pandas as pd
import numpy as np
import re

playerPage2 = requests.get("https://www.worldfootball.net/scorer/eng-premier-league-2019-2020/")
playerTree2 = html.fromstring(playerPage2.content)
x= 2
Name= []
Nation = []
Team = []
Goals= []
Points= []
Assists= []

while x < 70:
    teamCS = playerTree2.cssselect('tr')[x].text_content()
    tempPlayerlist = re.findall(r'[A-Z]\w+', teamCS)
    try:
        if tempPlayerlist[3] != ('City') and len(tempPlayerlist)<=5:
            tempName = (tempPlayerlist[0] + ' ' + tempPlayerlist[1])
            tempNationality= tempPlayerlist[2]
            tempTeam = ''
            for i in tempPlayerlist[3:]:
                tempTeam= (tempTeam +' ' + i)
            if len(re.findall(r'[.]', teamCS)) >0:
                tempPlayerStats = re.findall(r'\d+', teamCS)
                tempPoints = int(tempPlayerStats[1])
                tempGoals = int(tempPlayerStats[2])
                tempAssists = int(tempPlayerStats[3])
            else:
                tempPlayerStats = re.findall(r'\d+', teamCS)
                tempPoints = int(tempPlayerStats[0])
                tempGoals = int(tempPlayerStats[1])
                tempAssists = int(tempPlayerStats[2])
        elif tempPlayerlist[3] != ('City'):
            tempName = (tempPlayerlist[0] + ' ' + tempPlayerlist[1] + ' ' + tempPlayerlist[2])
            tempNationality= tempPlayerlist[3]
            tempTeam = ''
            for i in tempPlayerlist[4:]:
                tempTeam= (tempTeam +' ' + i)
            if len(re.findall(r'[.]', teamCS)) >0:
                tempPlayerStats = re.findall(r'\d+', teamCS)
                tempPoints = int(tempPlayerStats[1])
                tempGoals = int(tempPlayerStats[2])
                tempAssists = int(tempPlayerStats[3])
            else:
                tempPlayerStats = re.findall(r'\d+', teamCS)
                tempPoints = int(tempPlayerStats[0])
                tempGoals = int(tempPlayerStats[1])
                tempAssists = int(tempPlayerStats[2])
        else: 
            tempName = tempPlayerlist[0]
            tempNationality= tempPlayerlist[1]
            tempTeam = ''
            for i in tempPlayerlist[2:]:
                tempTeam= (tempTeam +' ' + i)
            if len(re.findall(r'[.]', teamCS)) >0:
                tempPlayerStats = re.findall(r'\d+', teamCS)
                tempPoints = int(tempPlayerStats[1])
                tempGoals = int(tempPlayerStats[2])
                tempAssists = int(tempPlayerStats[3])
            else:
                tempPlayerStats = re.findall(r'\d+', teamCS)
                tempPoints = int(tempPlayerStats[0])
                tempGoals = int(tempPlayerStats[1])
                tempAssists = int(tempPlayerStats[2])
    except IndexError:
        break

    
    x =x +1

    Name.append(tempName)
    Nation.append(tempNationality)
    Team.append(tempTeam)
    Points.append(tempPoints)
    Goals.append(tempGoals)
    Assists.append(tempAssists)

df = pd.DataFrame(
{'Name':Name,
 'Nation':Nation,
 'Team':Team,
 'Goals':Goals,
 'Assists':Assists,
 'Points':Points
})

#Show me the top 3 rows:

df.head()

Out[ 10 ]: 
 Assists Goals Name Nation Points Team
0 0 4 Raheem Sterling England 4 Manchester City
1 0 4 Teemu Pukki Finland 4 Norwich City
2 0 3 Ashley Barnes England 3 Burnley FC
3 1 2 Marcus Rashford England 3 Manchester United
4 3 0 Kevin De Bruyne Belgium 3 Manchester City

In [2]:
df = spark.createDataFrame(df)

In [3]:
spark.conf.set("fs.azure.createRemoteFileSystemDuringInitialization", "true")
dbutils.fs.ls("abfss://premierleague@taxicabdls.dfs.core.windows.net/")
spark.conf.set("fs.azure.createRemoteFileSystemDuringInitialization", "false")

--------------------------------------------------------------------------- 
 ExecutionError Traceback (most recent call last)
 <command-3569798548703904> in <module> () 
 1 spark . conf . set ( "fs.azure.createRemoteFileSystemDuringInitialization" , "true" ) 
 ----> 2 dbutils . fs . ls ( "abfss://premierleague@taxicabdls.dfs.core.windows.net/" ) 
 3 spark . conf . set ( "fs.azure.createRemoteFileSystemDuringInitialization" , "false" ) 

 /local_disk0/tmp/1566328870253-0/dbutils.py in f_with_exception_handling (*args, **kwargs) 
 302 exc . __context__ = None 
 303 exc . __cause__ = None 
 --> 304 raise exc 
 305 return f_with_exception_handling 
 306 

 ExecutionError : An error occurred while calling z:com.databricks.backend.daemon.dbutils.FSUtils.ls.
: Configuration property taxicabdls.dfs.core.windows.net not found.
	at shaded.databricks.v20180920_b33d810.org.apache.hadoop.fs.azurebfs.AbfsConfiguration.getStorageAccountKey(AbfsConfiguration.java:386)
	at shaded.databricks.v20180920_b33d810.org.apache.hadoop.fs.azurebfs.AzureBlobFileSystemStore.initializeClient(AzureBlobFileSystemStore.java:803)
	at shaded.databricks.v20180920_b33d810.org.apache.hadoop.fs.azurebfs.AzureBlobFileSystemStore.<init>(AzureBlobFileSystemStore.java:133)
	at shaded.databricks.v20180920_b33d810.org.apache.hadoop.fs.azurebfs.AzureBlobFileSystem.initialize(AzureBlobFileSystem.java:103)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:2669)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:370)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:295)
	at com.databricks.backend.daemon.dbutils.FSUtils$.getFS(DBUtilsCore.scala:251)
	at com.databricks.backend.daemon.dbutils.FSUtils$$anonfun$ls$1.apply(DBUtilsCore.scala:86)
	at com.databricks.backend.daemon.dbutils.FSUtils$$anonfun$ls$1.apply(DBUtilsCore.scala:85)
	at com.databricks.backend.daemon.dbutils.FSUtils$.com$databricks$backend$daemon$dbutils$FSUtils$$withFsSafetyCheck(DBUtilsCore.scala:81)
	at com.databricks.backend.daemon.dbutils.FSUtils$.ls(DBUtilsCore.scala:85)
	at com.databricks.backend.daemon.dbutils.FSUtils.ls(DBUtilsCore.scala)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)

In [4]:
spark.conf.set("fs.azure.account.key.taxicabdls.dfs.core.windows.net", "Z/5I6IAulN5jHvJ6ICxxlInKfuBo7EZ+4YdaEdFE3HsjPYHXemdx1sEeh7G0BLmft7MLtDWMOOxv4EoZ94XSfw==")

In [5]:
dbutils.fs.ls("abfss://premierleague@taxicabdls.dfs.core.windows.net/premierleague")

Out[ 21 ]: [FileInfo(path='abfss://premierleague@taxicabdls.dfs.core.windows.net/premierleague/sample.csv/', name='sample.csv/', size=0)]

In [6]:
df

Out[ 26 ]: DataFrame[Assists: bigint, Goals: bigint, Name: string, Nation: string, Points: bigint, Team: string]

In [7]:
df2 = spark.read.csv("abfss://premierleague@taxicabdls.dfs.core.windows.net/premierleague/sample2.csv", header=True, inferSchema=True)

In [8]:
spark.conf.set("fs.azure.sas.premierleague.taxicabdls.blob.core.windows.net", "Z/5I6IAulN5jHvJ6ICxxlInKfuBo7EZ+4YdaEdFE3HsjPYHXemdx1sEeh7G0BLmft7MLtDWMOOxv4EoZ94XSfw==")


In [9]:
df.coalesce(1).write.option("header", True).mode("overwrite").csv("abfss://premierleague@taxicabdls.dfs.core.windows.net/premierleague/sample2.csv")

In [10]:
spark.conf.set("fs.azure.sas.premierleague.taxicabdls.blob.core.windows.net", "Z/5I6IAulN5jHvJ6ICxxlInKfuBo7EZ+4YdaEdFE3HsjPYHXemdx1sEeh7G0BLmft7MLtDWMOOxv4EoZ94XSfw=="

output_container_path = "abfss://%s@%s.dfs.core.windows.net" % (CONTAINER, STORAGE_ACCOUNT)
output_blob_folder = "%s/premierleague" % output_container_path


df.coalesce(1).write.csv("mnt/.csv")






--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-3569798548703905> in <module> () 
 1 spark . conf . set ( "fs.azure.sas.premierleague.taxicabdls.blob.core.windows.net" , "Z/5I6IAulN5jHvJ6ICxxlInKfuBo7EZ+4YdaEdFE3HsjPYHXemdx1sEeh7G0BLmft7MLtDWMOOxv4EoZ94XSfw==" ) 
 2 
 ----> 3 output_container_path = "abfss://%s@%s.dfs.core.windows.net" % ( CONTAINER , STORAGE_ACCOUNT ) 
 4 output_blob_folder = "%s/premierleague" % output_container_path 
 5 

 NameError : name 'CONTAINER' is not defined

In [11]:

STORAGE_ACCOUNT = "taxicabdls"
CONTAINER = "premierleague"
MOUNT_POINT = "/mnt/premierleague"
SAS_KEY = "?sv=2018-03-28&ss=bfqt&srt=sco&sp=rwdlacup&se=2020-08-21T03:27:59Z&st=2019-08-20T19:27:59Z&spr=https&sig=nKMHVZuJyG%2BkadkbYfPSEydmB2CounQiU249e2YLrck%3D"
source_str = "abfss://{container}@{storage_acct}.dfs.core.windows.net/".format(container=CONTAINER, storage_acct=STORAGE_ACCOUNT)
conf_key = "fs.azure.sas.{container}.{storage_acct}.dfs.core.windows.net".format(container=CONTAINER, storage_acct=STORAGE_ACCOUNT)
spark.conf.set(
  "fs.azure.account.key.%s.dfs.core.windows.net" % STORAGE_ACCOUNT,  SAS_KEY)

output_container_path = "abfss://%s@%s.dfs.core.windows.net" % (CONTAINER, STORAGE_ACCOUNT)
output_blob_folder = "%s/premierleague" % output_container_path

try:
  dbutils.fs.mount(
    source = source_str,
    mount_point = MOUNT_POINT,
    extra_configs = {conf_key: SAS_KEY}
  )
except Exception as e:
  print("ERROR: {} already mounted. Run previous cells to unmount first".format(MOUNT_POINT))
  
  df.write.csv("abfss://premierleague@taxicabdls.dfs.core.windows.net/playstats.csv")